<a href="https://colab.research.google.com/github/thunwaaa/sign_language/blob/main/Sign__lang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ติดตั้งไลบรารีที่จำเป็น

In [ ]:
!pip install tensorflow opencv-python mediapipe pandas matplotlib scikit-learn

Then download an off-the-shelf model bundle. Check out the MediaPipe documentation for more information about this model bundle.

In [ ]:
!wget -q https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task

# นำเข้าไลบรารีที่จำเป็น

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from google.colab import drive
from mediapipe.python.solutions import hands as mp_hands
from mediapipe.framework.formats import landmark_pb2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

เชื่อมต่อ Google Drive เพื่อเข้าถึงข้อมูล

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
BASE_DIR = '/content/drive/MyDrive/sign'  # เปลี่ยนเป็นพาธที่เก็บข้อมูลของคุณ
model_path = '/content/drive/MyDrive/model'  # สำหรับบันทึกโมเดล

# สร้างโฟลเดอร์สำหรับบันทึกโมเดลถ้ายังไม่มี
if not os.path.exists(model_path):
    os.makedirs(model_path)

In [ ]:
# ตรวจสอบว่าโฟลเดอร์มีอยู่จริง
if os.path.exists(BASE_DIR):
    print(f"พบโฟลเดอร์: {BASE_DIR}")
    # ตรวจสอบจำนวนโฟลเดอร์ย่อย (คำในภาษามือ)
    subfolders = [f.name for f in os.scandir(BASE_DIR) if f.is_dir()]
    print(f"จำนวนคำในภาษามือ: {len(subfolders)}")
    print(f"ตัวอย่างคำ: {subfolders[:5] if len(subfolders) >= 5 else subfolders}")

    # ตรวจสอบจำนวนวิดีโอในโฟลเดอร์แรก
    if len(subfolders) > 0:
        first_folder = os.path.join(BASE_DIR, subfolders[0])
        videos = [f for f in os.listdir(first_folder) if f.endswith(('.mp4', '.avi', '.mov'))]
        print(f"จำนวนวิดีโอในโฟลเดอร์ '{subfolders[0]}': {len(videos)}")
else:
    print(f"ไม่พบโฟลเดอร์: {BASE_DIR}")
    print("โปรดตรวจสอบว่าคุณมีพาธที่ถูกต้องไปยังวิดีโอภาษามือไทยของคุณ")

## Visualization utilities

In [ ]:
def extract_hand_landmarks_from_video(video_path, max_frames=30):
    """
    ฟังก์ชันสำหรับสกัดจุดสำคัญของมือจากวิดีโอ (ปรับปรุงแล้ว)
    """
    cap = cv2.VideoCapture(video_path)
    frames_landmarks = []

    # ตั้งค่า MediaPipe Hands ด้วยความเชื่อมั่นที่ต่ำลง
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.3,  # ลดลงจาก 0.5
        min_tracking_confidence=0.3) as hands:  # ลดลงจาก 0.5

        # อ่านเฟรมทั้งหมดก่อน
        all_frames = []
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                break
            all_frames.append(image)

        # เลือกเฟรมแบบกระจาย
        if len(all_frames) > 0:
            selected_indices = np.linspace(0, len(all_frames)-1, min(max_frames, len(all_frames)), dtype=int)
            selected_frames = [all_frames[i] for i in selected_indices]

            # ประมวลผลเฟรมที่เลือก
            for image in selected_frames:
                if len(frames_landmarks) >= max_frames:
                    break

                # แปลงภาพจาก BGR เป็น RGB
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # ประมวลผลภาพเพื่อรู้จำมือ
                results = hands.process(image_rgb)

                # เก็บจุดสำคัญของมือ
                frame_landmarks = []
                if results.multi_hand_landmarks:
                    # ใช้มือแรกที่พบ
                    hand_landmarks = results.multi_hand_landmarks[0]

                    # สกัดพิกัด x, y, z ของจุดสำคัญทั้ง 21 จุด
                    for landmark in hand_landmarks.landmark:
                        frame_landmarks.extend([landmark.x, landmark.y, landmark.z])
                else:
                    # ถ้าไม่พบมือ ใส่ 0 สำหรับทุกพิกัด
                    frame_landmarks = [0.0] * (21 * 3)  # 21 จุด x 3 พิกัด

                frames_landmarks.append(frame_landmarks)

    cap.release()

    # ตรวจสอบว่าพบมือในวิดีโออย่างน้อย 1 เฟรมหรือไม่
    success = any(sum(landmarks) != 0 for landmarks in frames_landmarks)

    # ถ้าจำนวนเฟรมน้อยกว่า max_frames ให้เพิ่มเฟรมที่มีค่า 0 จนครบ
    while len(frames_landmarks) < max_frames:
        frames_landmarks.append([0.0] * (21 * 3))

    return np.array(frames_landmarks), success

    # ตรวจสอบวิดีโอใหม่
    word_folder = "ชื่อโฟลเดอร์ที่คุณเพิ่มวิดีโอ"  # เช่น "1.สวัสดี"
    video_file = "ชื่อไฟล์วิดีโอที่เพิ่ม"  # เช่น "สวัสดี_เพิ่ม.mp4"
    video_path = os.path.join(BASE_DIR, word_folder, video_file)

    if os.path.exists(video_path):
        print(f"กำลังตรวจสอบวิดีโอ: {video_path}")
        landmarks_sequence, success = extract_hand_landmarks_from_video(video_path)
        print(f"การตรวจจับมือสำเร็จ: {success}")
        if success:
            print(f"รูปร่างของข้อมูล: {landmarks_sequence.shape}")
    else:
        print(f"ไม่พบไฟล์: {video_path}")

ทดสอบฟังก์ชันสกัดจุดสำคัญกับวิดีโอตัวอย่าง

In [ ]:
if os.path.exists(BASE_DIR) and len(subfolders) > 0:
    # ค้นหาวิดีโอตัวอย่าง
    sample_folder = os.path.join(BASE_DIR, subfolders[0])
    sample_videos = [f for f in os.listdir(sample_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

    if sample_videos:
        sample_video_path = os.path.join(sample_folder, sample_videos[0])
        print(f"กำลังสกัดจุดสำคัญจากวิดีโอตัวอย่าง: {sample_video_path}")

        # สกัดจุดสำคัญ
        landmarks_sequence, success = extract_hand_landmarks_from_video(sample_video_path)

        print(f"การสกัดจุดสำคัญสำเร็จ: {success}")
        print(f"รูปร่างของลำดับจุดสำคัญ: {landmarks_sequence.shape}")
        print(f"จำนวนเฟรม: {landmarks_sequence.shape[0]}")
        print(f"จำนวนจุด (21 จุด x 3 พิกัด): {landmarks_sequence.shape[1]}")

        # แสดงตัวอย่างข้อมูลจากเฟรมแรก
        print("\nตัวอย่างข้อมูลจากเฟรมแรก (5 ค่าแรก):")
        print(landmarks_sequence[0][:15])  # แสดง 5 จุดแรก (x, y, z)
    else:
        print(f"ไม่พบวิดีโอในโฟลเดอร์ {subfolders[0]}")

ฟังก์ชันสำหรับสร้างชุดข้อมูลจากโฟลเดอร์



In [ ]:
def create_dataset_from_folders(base_dir, max_videos_per_folder=None):
    """
    สร้างชุดข้อมูลจากโฟลเดอร์ที่มีวิดีโอภาษามือไทย

    Args:
        base_dir: โฟลเดอร์หลักที่มีโฟลเดอร์ย่อยสำหรับแต่ละคำในภาษามือ
        max_videos_per_folder: จำนวนวิดีโอสูงสุดที่จะใช้จากแต่ละโฟลเดอร์ (ถ้าเป็น None จะใช้ทั้งหมด)

    Returns:
        X: ข้อมูลจุดสำคัญของมือ
        y: ป้ายกำกับ (คำในภาษามือ)
    """
    X = []
    y = []

    # วนลูปผ่านโฟลเดอร์ย่อยทั้งหมด
    for word_folder in tqdm(os.listdir(base_dir)):
        word_path = os.path.join(base_dir, word_folder)

        # ข้ามไฟล์ที่ไม่ใช่โฟลเดอร์
        if not os.path.isdir(word_path):
            continue

        # รวบรวมวิดีโอทั้งหมดในโฟลเดอร์
        videos = [f for f in os.listdir(word_path) if f.endswith(('.mp4', '.avi', '.mov'))]

        # จำกัดจำนวนวิดีโอต่อโฟลเดอร์ถ้ากำหนด
        if max_videos_per_folder is not None:
            videos = videos[:max_videos_per_folder]

        # วนลูปผ่านไฟล์วิดีโอที่เลือก
        for video_file in videos:
            video_path = os.path.join(word_path, video_file)

            try:
                # สกัดจุดสำคัญของมือจากวิดีโอ
                landmarks_sequence, success = extract_hand_landmarks_from_video(video_path)

                # เพิ่มข้อมูลเข้าชุดข้อมูลเฉพาะเมื่อพบมือในวิดีโอ
                if success:
                    X.append(landmarks_sequence)
                    y.append(word_folder)
                else:
                    print(f"ไม่พบมือในวิดีโอ: {video_path}")
            except Exception as e:
                print(f"เกิดข้อผิดพลาดในการประมวลผล {video_path}: {e}")

    return np.array(X), np.array(y)


# การสร้างและฝึกฝนโมเดล

### สร้างชุดข้อมูลขนาดเล็กสำหรับทดสอบ

In [ ]:
# สร้างชุดข้อมูลขนาดเล็กสำหรับทดสอบก่อน (ใช้เพียง 2 วิดีโอต่อโฟลเดอร์)
print("กำลังสร้างชุดข้อมูลขนาดเล็กสำหรับทดสอบ...")
max_folders_for_test = 5  # จำกัดเพียง 5 โฟลเดอร์แรก
max_videos_per_folder_for_test = 2  # จำกัดเพียง 2 วิดีโอต่อโฟลเดอร์

# สร้างโฟลเดอร์ย่อยชั่วคราวสำหรับทดสอบ
test_folders = subfolders[:max_folders_for_test] if len(subfolders) >= max_folders_for_test else subfolders

# แสดงโฟลเดอร์ที่จะใช้ในการทดสอบ
print(f"กำลังใช้ {len(test_folders)} โฟลเดอร์สำหรับการทดสอบ: {test_folders}")

# สร้างชุดข้อมูลขนาดเล็ก
X_test, y_test = create_dataset_from_folders(BASE_DIR, max_videos_per_folder=max_videos_per_folder_for_test)

print(f"ชุดข้อมูลทดสอบมี {len(X_test)} ตัวอย่าง")
print(f"รูปร่างของข้อมูล X: {X_test.shape}")
print(f"รูปร่างของข้อมูล y: {y_test.shape}")
print(f"คำในภาษามือที่พบในชุดข้อมูลทดสอบ: {np.unique(y_test)}")

In [ ]:
def debug_class_distribution(X, y):
    """
    ตรวจสอบการกระจายของข้อมูลตามคลาส (คำในภาษามือ)
    และแสดงคำที่มีความเสี่ยงจะถูกตัดออกเนื่องจากมีตัวอย่างน้อยเกินไป

    Args:
        X: ข้อมูลจุดสำคัญของมือ
        y: ป้ายกำกับ (คำในภาษามือ)
    """
    # คำนวณจำนวนตัวอย่างต่อคลาส
    unique_classes, counts = np.unique(y, return_counts=True)

    # สร้าง DataFrame เพื่อแสดงผลให้อ่านง่าย
    import pandas as pd
    class_distribution = pd.DataFrame({
        'คำ': unique_classes,
        'จำนวนตัวอย่าง': counts
    })

    # เรียงลำดับตามจำนวนตัวอย่าง (น้อยไปมาก)
    class_distribution = class_distribution.sort_values('จำนวนตัวอย่าง')

    # คำที่มีตัวอย่างน้อยกว่า 2 (จะถูกตัดออก)
    at_risk_classes = class_distribution[class_distribution['จำนวนตัวอย่าง'] < 2]

    # แสดงผลการวิเคราะห์
    print(f"จำนวนคำทั้งหมด: {len(unique_classes)}")
    print(f"คำที่เสี่ยงจะถูกตัดออก (มีตัวอย่างน้อยกว่า 2): {len(at_risk_classes)}")

    if len(at_risk_classes) > 0:
        print("\nรายการคำที่เสี่ยงจะถูกตัดออก:")
        print(at_risk_classes)

    # แสดงการกระจายของข้อมูลทั้งหมด
    print("\nการกระจายของข้อมูลทั้งหมด (เรียงจากน้อยไปมาก):")
    print(class_distribution)

    # วิเคราะห์จำนวนตัวอย่างเพิ่มเติม
    print(f"\nค่าเฉลี่ยจำนวนตัวอย่างต่อคำ: {counts.mean():.2f}")
    print(f"ค่ามัธยฐานจำนวนตัวอย่างต่อคำ: {np.median(counts)}")
    print(f"จำนวนตัวอย่างน้อยที่สุด: {counts.min()}")
    print(f"จำนวนตัวอย่างมากที่สุด: {counts.max()}")

    # คำที่มีจำนวนตัวอย่างน้อย (1-2 ตัวอย่าง)
    low_sample_classes = class_distribution[class_distribution['จำนวนตัวอย่าง'] <= 2]
    if len(low_sample_classes) > 0:
        print(f"\nคำที่มีตัวอย่างน้อย (1-2 ตัวอย่าง): {len(low_sample_classes)} คำ")
        print(low_sample_classes)

    # พล็อตกราฟแสดงการกระจายของจำนวนตัวอย่าง
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(counts)), sorted(counts))
    plt.xlabel('คำ (เรียงตามจำนวนตัวอย่าง)')
    plt.ylabel('จำนวนตัวอย่าง')
    plt.title('การกระจายของจำนวนตัวอย่างต่อคำ')
    plt.axhline(y=2, color='r', linestyle='--', label='ขีดแบ่งขั้นต่ำ (2 ตัวอย่าง)')
    plt.legend()
    plt.show()

    return at_risk_classes['คำ'].tolist()

# ใช้ฟังก์ชันเพื่อตรวจสอบคำที่จะถูกตัดออก
at_risk_words = debug_class_distribution(X_test, y_test)

print("\nสรุป: คำที่ต้องเพิ่มตัวอย่างเพื่อไม่ให้ถูกตัดออก:")
for word in at_risk_words:
    print(f"- {word}")

ตรวจสอบว่ามีไฟล์วิดีโอใดบ้างที่ไม่สามารถตรวจจับมือได้

In [ ]:
def check_video_detection_by_class(base_dir):
    """
    ตรวจสอบการตรวจจับมือในวิดีโอตามคลาส (คำในภาษามือ)

    Args:
        base_dir: โฟลเดอร์หลักที่มีโฟลเดอร์ย่อยสำหรับแต่ละคำในภาษามือ
    """
    results = {}

    # วนลูปผ่านโฟลเดอร์ย่อยทั้งหมด
    for word_folder in os.listdir(base_dir):
        word_path = os.path.join(base_dir, word_folder)

        # ข้ามไฟล์ที่ไม่ใช่โฟลเดอร์
        if not os.path.isdir(word_path):
            continue

        # รวบรวมวิดีโอทั้งหมดในโฟลเดอร์
        videos = [f for f in os.listdir(word_path) if f.endswith(('.mp4', '.avi', '.mov'))]

        success_count = 0
        fail_count = 0

        for video_file in videos:
            video_path = os.path.join(word_path, video_file)

            try:
                # สกัดจุดสำคัญของมือจากวิดีโอ
                _, success = extract_hand_landmarks_from_video(video_path)

                if success:
                    success_count += 1
                else:
                    fail_count += 1
                    print(f"ไม่พบมือในวิดีโอ: {video_path}")
            except Exception as e:
                fail_count += 1
                print(f"เกิดข้อผิดพลาดในการประมวลผล {video_path}: {e}")

        # เก็บผลลัพธ์
        results[word_folder] = {
            'total_videos': len(videos),
            'success': success_count,
            'fail': fail_count,
            'success_rate': success_count / len(videos) if len(videos) > 0 else 0
        }

    # แสดงผลลัพธ์
    import pandas as pd
    results_df = pd.DataFrame.from_dict(results, orient='index')
    results_df = results_df.sort_values('success_rate')

    print("\nผลการตรวจสอบการตรวจจับมือในวิดีโอตามคำในภาษามือ:")
    print(results_df)

    # แสดงคำที่มีปัญหามากที่สุด
    problem_classes = results_df[results_df['success'] == 0]
    if len(problem_classes) > 0:
        print("\nคำที่มีปัญหามากที่สุด (ไม่สามารถตรวจจับมือได้เลย):")
        print(problem_classes)

    return results_df

# ใช้ฟังก์ชันเพื่อตรวจสอบการตรวจจับมือในวิดีโอตามคำในภาษามือ
# โค้ดนี้อาจใช้เวลานานในการรัน ขึ้นอยู่กับจำนวนวิดีโอทั้งหมด
video_detection_results = check_video_detection_by_class(BASE_DIR)

### เตรียมข้อมูลสำหรับการฝึกฝนโมเดล

In [ ]:
def prepare_data(X, y):
    """
    เตรียมข้อมูลสำหรับการฝึกฝนโมเดล

    Args:
        X: ข้อมูลจุดสำคัญของมือ
        y: ป้ายกำกับ (คำในภาษามือ)

    Returns:
        X_train, X_val, X_test: ข้อมูลฝึกฝน, ตรวจสอบ, และทดสอบ
        y_train, y_val, y_test: ป้ายกำกับสำหรับแต่ละชุด
        num_classes: จำนวนคลาสทั้งหมด
        label_encoder: ตัวแปลงป้ายกำกับ
    """
    # แปลงป้ายกำกับเป็นตัวเลข
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # แบ่งข้อมูลเป็นชุดฝึกฝน, ตรวจสอบ, และทดสอบ
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, stratify=y_train_val)

    # แปลงป้ายกำกับเป็นเวกเตอร์ one-hot
    num_classes = len(label_encoder.classes_)
    y_train = to_categorical(y_train, num_classes)
    y_val = to_categorical(y_val, num_classes)
    y_test = to_categorical(y_test, num_classes)

    print(f"รูปร่างของข้อมูลฝึกฝน: {X_train.shape}")
    print(f"รูปร่างของข้อมูลตรวจสอบ: {X_val.shape}")
    print(f"รูปร่างของข้อมูลทดสอบ: {X_test.shape}")
    print(f"จำนวนคลาส: {num_classes}")
    print(f"คลาสของคำในภาษามือ: {label_encoder.classes_}")

    return X_train, X_val, X_test, y_train, y_val, y_test, num_classes, label_encoder

# เตรียมข้อมูลจากชุดข้อมูลทดสอบ
X_train, X_val, X_test, y_train, y_val, y_test, num_classes, label_encoder = prepare_data(X_test, y_test)

### สร้างโมเดล LSTM สำหรับการรู้จำภาษามือ

In [ ]:
def create_model(input_shape, num_classes):
    """
    สร้างโมเดลสำหรับการรู้จำภาษามือไทย

    Args:
        input_shape: รูปร่างของข้อมูลอินพุต
        num_classes: จำนวนคลาสทั้งหมด

    Returns:
        model: โมเดล Keras
    """
    model = Sequential([
        LSTM(64, return_sequences=True, activation='relu', input_shape=input_shape),
        Dropout(0.3),
        LSTM(32, return_sequences=False, activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print(model.summary())
    return model

# สร้างโมเดล
input_shape = (X_train.shape[1], X_train.shape[2])  # (frames, features)
model = create_model(input_shape, num_classes)

### ฝึกฝนโมเดล

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=50):
    """
    ฝึกฝนโมเดลการรู้จำภาษามือไทย

    Args:
        model: โมเดลที่สร้างขึ้น
        X_train, y_train: ข้อมูลฝึกฝนและป้ายกำกับ
        X_val, y_val: ข้อมูลตรวจสอบและป้ายกำกับ
        epochs: จำนวนรอบในการฝึกฝน

    Returns:
        history: ประวัติการฝึกฝน
    """
    # กำหนด callbacks
    callbacks = [
        EarlyStopping(patience=10, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.1, patience=5),
        ModelCheckpoint('thai_sign_language_model.h5', save_best_only=True)
    ]

    # ฝึกฝนโมเดล
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=16,
        callbacks=callbacks
    )

    return history

# ฝึกฝนโมเดล - ใช้เพียง 10 epochs สำหรับการทดสอบ
history = train_model(model, X_train, y_train, X_val, y_val, epochs=10)

### แสดงประวัติการฝึกฝน

In [ ]:
def plot_training_history(history):
    """
    แสดงกราฟประวัติการฝึกฝน

    Args:
        history: ประวัติการฝึกฝนจาก model.fit()
    """
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ความแม่นยำของโมเดล')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='lower right')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('การสูญเสียของโมเดล')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')

    plt.tight_layout()
    plt.show()

# แสดงกราฟประวัติการฝึกฝน
plot_training_history(history)

#การทดสอบและประเมินผลโมเดล

### ประเมินผลโมเดลบนชุดข้อมูลทดสอบ

In [ ]:
def evaluate_model(model, X_test, y_test, label_encoder):
    """
    ประเมินผลโมเดลบนชุดข้อมูลทดสอบ

    Args:
        model: โมเดลที่ฝึกฝนแล้ว
        X_test, y_test: ข้อมูลทดสอบและป้ายกำกับ
        label_encoder: ตัวแปลงป้ายกำกับ
    """
    # คำนวณความแม่นยำบนชุดข้อมูลทดสอบ
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"ความแม่นยำบนชุดข้อมูลทดสอบ: {test_accuracy:.4f}")

    # ทำนายบนชุดข้อมูลทดสอบ
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # แปลงกลับเป็นป้ายกำกับเดิม
    y_pred_labels = label_encoder.inverse_transform(y_pred_classes)
    y_true_labels = label_encoder.inverse_transform(y_true_classes)

    # แสดงตัวอย่างการทำนาย
    print("\nตัวอย่างการทำนาย:")
    for i in range(min(len(y_pred_labels), 5)):
        print(f"ตัวอย่างที่ {i+1}: คำทำนาย = {y_pred_labels[i]}, คำจริง = {y_true_labels[i]}")

# ประเมินผลโมเดล
evaluate_model(model, X_test, y_test, label_encoder)

### บันทึกโมเดลและตัวแปลงป้ายกำกับ

In [ ]:
model.save('thai_sign_language_model_test.h5')
np.save('label_encoder_classes_test.npy', label_encoder.classes_)

print("บันทึกโมเดลและตัวแปลงป้ายกำกับเรียบร้อยแล้ว")

### ฟังก์ชันสำหรับการใช้โมเดลกับกล้องเรียลไทม์

In [ ]:
def visualize_landmarks(image, results):
    """
    วาดจุดสำคัญของมือบนภาพ

    Args:
        image: ภาพที่จะวาด
        results: ผลลัพธ์จาก MediaPipe Hands

    Returns:
        image: ภาพที่วาดจุดสำคัญแล้ว
    """
    # วาดจุดสำคัญของมือ
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

    return image

def realtime_sign_recognition(model, label_encoder):
    """
    การรู้จำภาษามือแบบเรียลไทม์จากกล้อง

    Args:
        model: โมเดลที่ฝึกฝนแล้ว
        label_encoder: ตัวแปลงป้ายกำกับ
    """
    cap = cv2.VideoCapture(0)

    # ตั้งค่า MediaPipe Hands
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        # เก็บลำดับของจุดสำคัญ
        sequence = []
        predictions = []
        threshold = 0.5

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("ไม่สามารถเปิดกล้องได้")
                break

            # ตั้งค่าภาพเพื่อการแสดงผล
            image = cv2.flip(image, 1)  # กลับด้านซ้าย-ขวา
            display_image = image.copy()

            # แปลงภาพสำหรับ MediaPipe
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image_rgb)

            # วาดจุดสำคัญบนภาพ
            display_image = visualize_landmarks(display_image, results)

            # สกัดจุดสำคัญของมือ
            frame_landmarks = []
            if results.multi_hand_landmarks:
                # ใช้มือแรกที่พบ
                hand_landmarks = results.multi_hand_landmarks[0]

                # สกัดพิกัด x, y, z
                for landmark in hand_landmarks.landmark:
                    frame_landmarks.extend([landmark.x, landmark.y, landmark.z])
            else:
                # ถ้าไม่พบมือ ใส่ 0
                frame_landmarks = [0.0] * (21 * 3)

            # เพิ่มเฟรมเข้าลำดับ
            sequence.append(frame_landmarks)
            sequence = sequence[-30:]  # เก็บแค่ 30 เฟรมล่าสุด

            # ทำนายเมื่อมีเฟรมครบ 30 เฟรม
            if len(sequence) == 30:
                # เตรียมข้อมูลสำหรับทำนาย
                X = np.expand_dims(np.array(sequence), axis=0)

                # ทำนาย
                prediction = model.predict(X)[0]
                predicted_class = np.argmax(prediction)
                confidence = prediction[predicted_class]

                # เพิ่มการทำนายเข้าในรายการถ้าความเชื่อมั่นสูงกว่าขีดแบ่ง
                if confidence > threshold:
                    predicted_word = label_encoder.inverse_transform([predicted_class])[0]
                    predictions.append(predicted_word)

                    # แสดงคำที่ปรากฏบ่อยที่สุดใน 5 การทำนายล่าสุด
                    if len(predictions) > 0:
                        # นับความถี่
                        from collections import Counter
                        counter = Counter(predictions[-5:])
                        most_common = counter.most_common(1)[0][0]

                        # แสดงผลคำที่ทำนาย
                        cv2.putText(display_image, f"{most_common} ({confidence:.2f})",
                                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # แสดงภาพ
            cv2.imshow('Thai Sign Language Recognition', display_image)

            # กด 'q' เพื่อออก
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

### ทดสอบการทำนายด้วยวิดีโอตัวอย่าง

In [ ]:
def predict_from_sample_video(model, label_encoder, video_path):
    """
    ทดสอบการทำนายจากวิดีโอตัวอย่าง

    Args:
        model: โมเดลที่ฝึกฝนแล้ว
        label_encoder: ตัวแปลงป้ายกำกับ
        video_path: พาธของไฟล์วิดีโอ
    """
    # สกัดจุดสำคัญของมือจากวิดีโอ
    landmarks_sequence, success = extract_hand_landmarks_from_video(video_path)

    if not success:
        print("ไม่พบมือในวิดีโอ")
        return

    # เพิ่มมิติแรกเพื่อให้สอดคล้องกับรูปแบบอินพุตของโมเดล
    X = np.expand_dims(landmarks_sequence, axis=0)

    # ทำนาย
    prediction = model.predict(X)[0]
    predicted_class = np.argmax(prediction)
    confidence = prediction[predicted_class]

    # แปลงกลับเป็นคำ
    predicted_word = label_encoder.inverse_transform([predicted_class])[0]

    print(f"คำที่ทำนาย: {predicted_word}")
    print(f"ความเชื่อมั่น: {confidence:.4f}")

    # แสดงผลความเชื่อมั่นสำหรับทุกคลาส
    print("\nความเชื่อมั่นสำหรับทุกคลาส:")
    for i, conf in enumerate(prediction):
        word = label_encoder.inverse_transform([i])[0]
        print(f"{word}: {conf:.4f}")

# ถ้ามีวิดีโอตัวอย่าง ให้ทดสอบการทำนาย
if os.path.exists(BASE_DIR) and len(subfolders) > 0:
    # ค้นหาวิดีโอตัวอย่าง
    sample_folder = os.path.join(BASE_DIR, subfolders[0])
    sample_videos = [f for f in os.listdir(sample_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

    if sample_videos:
        sample_video_path = os.path.join(sample_folder, sample_videos[0])
        print(f"กำลังทดสอบการทำนายจากวิดีโอตัวอย่าง: {sample_video_path}")
        predict_from_sample_video(model, label_encoder, sample_video_path)

## เตรียมฟังก์ชันสำหรับการฝึกฝนเต็มรูปแบบ

In [ ]:
def create_full_dataset(base_dir, max_videos_per_folder=None, max_folders=None):
    """
    สร้างชุดข้อมูลเต็มรูปแบบจากโฟลเดอร์ทั้งหมด

    Args:
        base_dir: โฟลเดอร์หลัก
        max_videos_per_folder: จำนวนวิดีโอสูงสุดต่อโฟลเดอร์ (None คือใช้ทั้งหมด)
        max_folders: จำนวนโฟลเดอร์สูงสุดที่จะใช้ (None คือใช้ทั้งหมด)

    Returns:
        X, y: ข้อมูลและป้ายกำกับ
    """
    # รวบรวมโฟลเดอร์ทั้งหมด
    all_folders = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]

    # จำกัดจำนวนโฟลเดอร์ถ้ากำหนด
    if max_folders is not None:
        all_folders = all_folders[:max_folders]

    # จำลองโฟลเดอร์ย่อยด้วยโฟลเดอร์ที่เลือก
    base_dir_temp = base_dir

    # สร้างชุดข้อมูล
    print(f"กำลังสร้างชุดข้อมูลจาก {len(all_folders)} โฟลเดอร์...")
    X, y = create_dataset_from_folders(base_dir_temp, max_videos_per_folder)

    return X, y

def full_training_pipeline(base_dir, max_videos_per_folder=None, max_folders=None, epochs=50):
    """
    กระบวนการฝึกฝนเต็มรูปแบบ

    Args:
        base_dir: โฟลเดอร์หลัก
        max_videos_per_folder: จำนวนวิดีโอสูงสุดต่อโฟลเดอร์
        max_folders: จำนวนโฟลเดอร์สูงสุดที่จะใช้
        epochs: จำนวนรอบในการฝึกฝน

    Returns:
        model: โมเดลที่ฝึกฝนแล้ว
        label_encoder: ตัวแปลงป้ายกำกับ
    """
    # 1. สร้างชุดข้อมูล
    X, y = create_full_dataset(base_dir, max_videos_per_folder, max_folders)

    # 2. เตรียมข้อมูล
    X_train, X_val, X_test, y_train, y_val, y_test, num_classes, label_encoder = prepare_data(X, y)

    # 3. สร้างและฝึกฝนโมเดล
    input_shape = (X_train.shape[1], X_train.shape[2])
    model = create_model(input_shape, num_classes)
    history = train_model(model, X_train, y_train, X_val, y_val, epochs)

    # 4. แสดงประวัติการฝึกฝน
    plot_training_history(history)

    # 5. ประเมินผลโมเดล
    evaluate_model(model, X_test, y_test, label_encoder)

    # 6. บันทึกโมเดลและตัวแปลงป้ายกำกับ
    model.save('thai_sign_language_model_full.h5')
    np.save('label_encoder_classes_full.npy', label_encoder.classes_)

    print("บันทึกโมเดลและตัวแปลงป้ายกำกับเรียบร้อยแล้ว")

    return model, label_encoder

# ฝึกฝนโมเดลเต็มรูปแบบ **(รันเมื่อพร้อมเท่านั้น)**

In [ ]:
"""
# คำเตือน: การรันเซลล์นี้จะใช้เวลานานมาก ขึ้นอยู่กับปริมาณข้อมูลของคุณ

# จำกัดจำนวนวิดีโอและโฟลเดอร์เพื่อลดเวลาในการฝึกฝน
# ถ้าต้องการใช้ทั้งหมด ให้ตั้งค่าเป็น None
max_videos = 3  # จำนวนวิดีโอสูงสุดต่อโฟลเดอร์
max_folders = 10  # จำนวนโฟลเดอร์สูงสุด
epochs = 30  # จำนวนรอบในการฝึกฝน

# ฝึกฝนโมเดลเต็มรูปแบบ
full_model, full_label_encoder = full_training_pipeline(BASE_DIR, max_videos, max_folders, epochs)
"""

## โหลดโมเดลที่ฝึกฝนไว้แล้ว

In [ ]:
def load_model_and_classes(model_path='thai_sign_language_model_test.h5', classes_path='label_encoder_classes_test.npy'):
    """
    โหลดโมเดลและคลาสที่บันทึกไว้

    Args:
        model_path: พาธของไฟล์โมเดล
        classes_path: พาธของไฟล์คลาส

    Returns:
        model: โมเดลที่โหลด
        label_encoder: ตัวแปลงป้ายกำกับ
    """
    # โหลดโมเดล
    model = tf.keras.models.load_model(model_path)

    # โหลดคลาส
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load(classes_path, allow_pickle=True)

    print(f"โหลดโมเดลจาก {model_path} สำเร็จ")
    print(f"จำนวนคลาส: {len(label_encoder.classes_)}")
    print(f"คลาสทั้งหมด: {label_encoder.classes_}")

    return model, label_encoder

# ตรวจสอบว่าโมเดลถูกสร้างไว้แล้วหรือไม่
if os.path.exists('thai_sign_language_model_test.h5'):
    # โหลดโมเดลที่ฝึกฝนไว้แล้ว
    loaded_model, loaded_label_encoder = load_model_and_classes()

# การใช้โมเดลกับกล้องเรียลไทม์

In [ ]:
"""
# คำเตือน: เซลล์นี้จะเปิดกล้องของคุณและใช้โมเดลที่ฝึกฝนแล้วในการรู้จำภาษามือ
# รันเฉพาะเมื่อคุณพร้อมที่จะทดสอบกับกล้อง

# ใช้โมเดลที่ฝึกฝนไว้แล้ว ทำงานกับกล้อง
if 'loaded_model' in locals() and 'loaded_label_encoder' in locals():
    print("กำลังเริ่มการรู้จำภาษามือแบบเรียลไทม์...")
    print("กด 'q' เพื่อออกจากโปรแกรม")
    realtime_sign_recognition(loaded_model, loaded_label_encoder)
else:
    print("โปรดฝึกฝนหรือโหลดโมเดลก่อนใช้งานกับกล้อง")
"""

# **โค้ดสำหรับการโหลดและใช้โมเดลที่ฝึกฝนไว้แล้ว**

In [ ]:
def load_model_and_recognize():
    """
    โหลดโมเดลที่ฝึกฝนไว้แล้วและใช้ในการรู้จำภาษามือแบบเรียลไทม์
    """
    # โหลดโมเดล
    model = tf.keras.models.load_model('thai_sign_language_model.h5')

    # โหลดตัวแปลงป้ายกำกับ
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load('label_encoder_classes.npy', allow_pickle=True)

    # ใช้โมเดลกับกล้องเรียลไทม์
    realtime_sign_recognition(model, label_encoder)

# เรียกใช้เมื่อต้องการใช้โมเดลที่ฝึกฝนไว้แล้ว
# load_model_and_recognize()